In [58]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import ConfigParser
import os

#read in the config file
config = ConfigParser.ConfigParser()

#print os.path.isfile() 
config.read(os.getcwd() + "/Infra_2/capstone_config.ini")

ES_url = config.get('ElasticSearch','host')
ES_password = config.get('ElasticSearch','password')
ES_username= config.get('ElasticSearch','username')
#es = Elasticsearch(timeout=30)
#es = Elasticsearch(timeout=30, max_retries=10, retry_on_timeout=True)
es = Elasticsearch(['http://' + ES_username + ':' + ES_password + '@' + ES_url + ':9200/'],timeout=60)

# Getting list of all of the zipcodes in NYC

In [88]:
collisions = helpers.scan(es,
    query={"query": {"match_all": {}}},
    index="nyc_zip_codes",
    doc_type="zip_codes"
)



index = 0

zipc=[]
for collision in collisions:
    #print collision
    #print collision["_source"]["GEOID10"]
    #break
    zipc.append(int(collision["_source"]["GEOID10"]))    
   
    index +=1
    
#since we will get duplicates (we split each polygon in a multipolygon zipcode to one row each)
#clean up duplicates
uniqueZipFromZipCode =list(set(zipc))
print len(uniqueZipFromZipCode)
print ",".join(map(str,uniqueZipFromZipCode))
#print index

213
10271,10278,10279,10280,10282,10803,10301,10302,10303,10304,10305,10306,10307,10308,10309,10310,10311,10312,10314,11351,11354,11355,11356,11357,11358,11359,11360,11361,11362,11363,11364,11365,11366,11367,11368,11369,11370,11371,11372,11373,11374,11375,11377,11378,11379,11385,11201,11411,11412,11413,11414,11415,11416,11417,11418,11419,11420,11421,11422,11423,11424,11425,11426,11427,11428,11429,11430,11432,11433,11434,11435,11436,11451,11040,10451,10452,10453,10454,10455,10456,10457,10458,10459,10460,10461,10462,10463,10464,10465,10466,10467,10468,10469,10470,10471,10472,10473,10474,10475,11001,11004,11005,10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10016,10017,10018,10019,10020,10021,10022,10023,10024,10025,10026,10027,10028,10029,10030,10031,10032,10033,10034,10035,10036,10037,10038,10039,10040,10044,10065,10069,10075,11101,11102,11103,11104,11105,11106,11109,10103,10110,10111,10112,10115,10119,10128,10152,10153,10154,11691,11692,11693,11694,11697,

# Get all zip codes in collision data 

In [85]:
collisions = helpers.scan(es,
    query={"query": {"match_all": {}}},
    index="saferoad",
    doc_type="collisions"
)

index = 0

zipcFromCollisions=[]
for collision in collisions:
    #print collision
    zc = collision["_source"]["collision_ZCTA_ZIP_NoSuffix"]
    if zc != "NA":        
        zipcFromCollisions.append(int(collision["_source"]["collision_ZCTA_ZIP_NoSuffix"]))    
   
    #index +=1
    #if index == 20:
    #   break
    
#clean up duplicates
uniqueZipFromCollisions =list(set(zipcFromCollisions))
#print len(uniqueZip)
#print ",".join(map(str,uniqueZip))
##print index

In [89]:
for zc in uniqueZipFromCollisions:
    if not zc in uniqueZipFromZipCode:
        print zc


# In collisions index, reindex zip code with column name without space

In [34]:
indexName = "saferoad"
docType = "collisions"

collisions = helpers.scan(es,
    query={"query": {"match_all": {}}},
    index=indexName,
    doc_type=docType
)

bulkIncrement = 1000

bulkCounter= 0


bulkPackage = []
for sdf in collisions:
    
    zc = sdf["_source"]["collision_ZIP CODE"]
       
    if len(zc) == 0:
        zc = "NA"
    else
    
    bulkPackage.append({
        '_op_type': 'update',
        '_index': indexName,
        '_type': docType,
        '_id': sdf["_id"],
        'doc': {'collision_ZIP_CODE_C': zc}
    })
    

    #es.update(index='saferoad',doc_type='collisions',id=sdf["_id"],
    #            body={"doc": {"DateTime": concatDate(sdf["_source"]["Date"],sdf["_source"]["Time"])}})
 
    
    
    
    #bulkCounter +=1
    #if bulkCounter == bulkIncrement: #update
    #    helpers.bulk(es, bulkPackage)
        #reset 
    #    bulkPackage = []
    #    bulkCounter = 0
        
    
        
if len(bulkPackage) != 0:
    helpers.bulk(es, bulkPackage)

# Merge collision data to the grid. 
### We are marking 1) count of accidents given zipcode & hour 2) accident (1) or not (0)

In [90]:
import datetime

indexName = "saferoad"
docType = "collisions"

collisions = helpers.scan(es,
    query={"query": {"match_all": {}}},
    index=indexName,
    doc_type=docType
)

#id in this format
#2014-05-09T16:00:00_11005
#2014-07-22T17:02:00

#isProbZipcode = dict()
#isProbZipcode["10281"] = 0
#isProbZipcode["10000"] = 0
#isProbZipcode["10048"] = 0
#isProbZipcode["11695"] = 0
#isProbZipcode["11249"] = 0

dateZiplist = dict()
index = 0



for col in collisions:
    zipCode = col["_source"]["collision_ZCTA_ZIP_NoSuffix"].encode('ascii','ignore')
    if zipCode != "NA":
        
        dateTimeStr = col["_source"]["collision_DATETIME_C"].encode('ascii','ignore')
        #print dateTimeStr
        accidentdt = datetime.datetime.strptime(dateTimeStr, '%Y-%m-%dT%H:%M:%S')        
        hourMark = accidentdt.replace(minute=0, second=0)                
        dateAndZip = hourMark.strftime('%Y-%m-%dT%H:%M:%S') + "_" + zipCode
        #print dateAndZip
        if dateAndZip in dateZiplist: #accident was already recorded at this location at this hour
            #append to existing records
            dateZiplist[dateAndZip]["grid_collision_counter"] += 1
            #dateZiplist[dateAndZip]["collisionRec_" + col["_id"]] =  1
        else:
            #insert new record
            dateZiplist[dateAndZip] = dict()
            dateZiplist[dateAndZip]["grid_collision_counter"] = 1
            #dateZiplist[dateAndZip]["grid_collisionRec_" + col["_id"]] =  1
            dateZiplist[dateAndZip]["grid_isAccident"] =  1
            

    
#helpers.bulk(es, bulkGridPackage,chunk_size=300)        


# based on the dict created above, put together bulk update package

In [92]:
#building bulk package 

bulkGridPackage = []        
for key, value in dateZiplist.iteritems():
    #print key
    
#    es.update(index='weather_observations',doc_type='weather_observations',id=sdf["_id"],
#        body={"doc": {"DateTime": concatDate(sdf["_source"]["Date"],sdf["_source"]["Time"])}})


    bulkGridPackage.append({
        '_op_type': 'update',
        '_index': "dataframe",
        '_type': "rows",
        '_id': key,
        'doc': value
    })
 

# Couting how many times 5 zip codes in collision that never made it into master zip code list show up in collisions (OLD)

In [40]:
for key, value in isProbZipcode.iteritems():
    print "zipcode:" + key + " count:" + str(value) 


zipcode:10048 count:29
zipcode:11249 count:1955
zipcode:11695 count:10
zipcode:10000 count:7
zipcode:10281 count:30


# checking to see if all of the accients exist in the grid
### it seems an hour that was supposed to be "lost" when DST kicked in existed in the collision dataset (data entry error)

In [42]:
index = 0
for key, value in dateZiplist.iteritems():
    #print key
    #print value
    index+=1
    #break
    row = es.search(index="dataframe",doc_type='rows', size=1, \
                       body={"query": \
                                {"match": \
                                    {'_id':key} \
                                } \
                            })
    
    #print key
    hits =  len(row["hits"]["hits"])
    #break  
    if hits == 0:
        print key

    #if index == 1000:
    #    break

2014-03-09T02:00:00_11203
2014-03-09T02:00:00_11414
2013-03-10T02:00:00_11225


# insert collisions to the grid 

In [94]:

helpers.bulk(es, bulkGridPackage,raise_on_error=False)  

(570174,
 [{u'update': {u'_id': u'2013-03-10T02:00:00_11001',
    u'_index': u'dataframe',
    u'_type': u'rows',
    u'error': {u'index': u'dataframe',
     u'reason': u'[rows][2013-03-10T02:00:00_11001]: document missing',
     u'shard': u'-1',
     u'type': u'document_missing_exception'},
    u'status': 404}},
  {u'update': {u'_id': u'2014-03-09T02:00:00_11203',
    u'_index': u'dataframe',
    u'_type': u'rows',
    u'error': {u'index': u'dataframe',
     u'reason': u'[rows][2014-03-09T02:00:00_11203]: document missing',
     u'shard': u'-1',
     u'type': u'document_missing_exception'},
    u'status': 404}},
  {u'update': {u'_id': u'2014-03-09T02:00:00_11414',
    u'_index': u'dataframe',
    u'_type': u'rows',
    u'error': {u'index': u'dataframe',
     u'reason': u'[rows][2014-03-09T02:00:00_11414]: document missing',
     u'shard': u'-1',
     u'type': u'document_missing_exception'},
    u'status': 404}},
  {u'update': {u'_id': u'2013-03-10T02:00:00_11214',
    u'_index': u'dat

# Adding month and day to the grid (NOTE: not used anymore b/c it was added in the grid creation stage)


In [59]:
import datetime

rows = helpers.scan(es,
    query={"query": {"match_all": {}}},
    index="dataframe",
    doc_type="rows"
)

index = 0

addMonthDayPackage=[]
for row in rows:
    #print row
    #using _id row instead of grid_fullDate b/c grid_fullDate has timezone nightmare
    thisDateTime =row["_id"].split("_")[0]

    rowDt = datetime.datetime.strptime(thisDateTime, '%Y-%m-%dT%H:%M:%S') 
    #print rowDt.month
    #print rowDt.day
    
    addMonthDayPackage.append({
        '_op_type': 'update',
        '_index': "dataframe",
        '_type': "rows",
        '_id': row["_id"],
        'doc': {'grid_month': rowDt.month, 'grid_day': rowDt.day}
    })
 
        
    #zc = collision["_source"]["collision_ZIP_CODE_C"]


In [61]:
helpers.bulk(es, addMonthDayPackage,raise_on_error=False)  

(6968370, [])

# Remove "-" from ZCTA zip codes and add as a new column

In [82]:
collisions = helpers.scan(es,
    query={"query": {"match_all": {}}},
    index="saferoad",
    doc_type="collisions"
)

index = 0

zipcFromZCTA=[]
for collision in collisions:
    #print collision
    zc = collision["_source"]["collision_ZCTA_ZIP"]
    
    if not zc is None:        
        if "-" in zc: #remove -1, -2 etc        
            zc = zc.split("-")[0]   
            
    else: #code None as NA as well
        zc = "NA"

    zipcFromZCTA.append({
        '_op_type': 'update',
        '_index': "saferoad",
        '_type': "collisions",
        '_id': collision["_id"],
        'doc': {'collision_ZCTA_ZIP_NoSuffix': zc }
    })
    
  

In [84]:
helpers.bulk(es, zipcFromZCTA,raise_on_error=False)  

(735535, [])